Todo:

- batenkaart
- kwetsbaarheden

In [2]:
from ipywidgets import interactive
from IPython.display import Audio, display
import numpy as np
import matplotlib.pyplot as plt
def beat_freq(f1=220.0, f2=224.0):
    max_time = 3
    rate = 8000
    times = np.linspace(0,max_time,rate*max_time)
    signal = np.sin(2*np.pi*f1*times) + np.sin(2*np.pi*f2*times)
    display(Audio(data=signal, rate=rate))
    
    return signal

v = interactive(beat_freq, f1=(200.0,300.0), f2=(200.0,300.0))
f1, f2 = v.children[:2]
f1.value = 255
f2.value = 260
plt.plot(v.result[0:6000])

In [2]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [1]:
# %% VOORAF
# Zorg dat in de poldermap onder 01. DAMO en HDB een mapje staat met de naam 'peilgebieden'. 
# In dit mapje mapje moet de shape van de peilgebieden. Deze shape moet 'peilgebieden_Polderclusternaam.shp' heten, 
# dus bijvoorbeeld 'peilgebieden_Eijerland.shp'. In de kolom 'name' moet de naam van de polder staan. 
# Dit laatste is van belang wanneer meerdere polders in één poldercluster gezamelijk zijn doorgerekend. 
# Maak een kopie van je datachecker output: fixeddrainagelevelarea. Dat werkt.


# %%
import sys

from numpy.lib.function_base import kaiser
# sys.path.append(r'C:/Users/chris.kerklaan/Documents/Github/hhnk-threedi-tools')
sys.path.append('C:\\Users\wvangerwen\github\hhnk-threedi-tools')

from hhnk_threedi_tools import Folders
import hhnk_threedi_tools as htt
import hhnk_research_tools as hrt


# import hhnk_threedi_tools.core.climate_scenarios as hrt_climate
import hhnk_threedi_tools.core.climate_scenarios.maskerkaart as maskerkaart
import hhnk_threedi_tools.core.climate_scenarios.ruimtekaart as ruimtekaart
from hhnk_threedi_tools.core.climate_scenarios.interpolate_rasters import main_interpolate_rasters
import hhnk_threedi_tools.core.climate_scenarios.schadekaart as schadekaart
import hhnk_threedi_tools.core.climate_scenarios.peilgebieden as peilgebieden



from hhnk_threedi_tools.core.climate_scenarios.masker_filter_rasters import rasterize_maskerkaart, apply_mask_to_raster, remove_mask_from_raster

import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from osgeo import gdal


# %matplotlib inline

import importlib.resources as pkg_resources #Load resource from package
import ipywidgets as widgets

import sys, importlib
import hhnk_threedi_tools.utils.notebooks.rasterclass as rasterclass
importlib.reload(rasterclass)

#User input
# %%
# --------------------------------------------------
polders_folder = "C:/Users/wvangerwen/Github/hhnk-threedi-tools/hhnk_threedi_tools/tests/data/multiple_polders"
polder = 'egmond' 
# --------------------------------------------------

folder = Folders(os.path.join(polders_folder, polder))


def item_layout(width="95%", grid_area="", **kwargs):
    return widgets.Layout(
        width=width, grid_area=grid_area, **kwargs
    )  # override the default width of the button to 'auto' to let the button grow
    
output_folder_options = folder.threedi_results.batch.content
# %%
output_folder_label = widgets.Label('Selecteer output folder:', layout=item_layout(grid_area='output_folder_label'))
output_folder_box = widgets.Select(
    options=output_folder_options,
    rows=3,
    disabled=False,
    layout=item_layout(grid_area="output_folder_box"),
)

In [1]:
#Import all packages. GDAL should be installed including BIGTIFF option
# import getpass
# import pandas as pd
# import os
# from threedi_scenario_downloader import downloader as dl
# from IPython.display import display, HTML
# import shutil
# import logging
# import geopandas as gpd
# import numpy as np
# from ipywidgets import FloatProgress
# import matplotlib.pyplot as plt
# import shapely.geometry
# import gc #garbage collection -> free memory




# import functions_nabewerking.maskerkaart as maskerkaart    #script should be in the same folder as the Python file/notebook
# import functions_nabewerking.ruimtekaart as ruimtekaart  #script should be in the same folder as the Python file/notebook
# from functions_nabewerking.create_batch_folders_dict import create_batch_folders_dict
# from functions_nabewerking.interpoleer_rasters import main_interpolate_rasters_parallel, interpoleer_raster_parallel, main_interpolate_rasters, interpoleer_raster, interpoleer_deel, lees_waterdieptes, bepaal_delen
# from functions_nabewerking.masker_filtering_rasters import rasterize_maskerkaart, apply_mask_to_raster, remove_mask_from_raster
# from functions_nabewerking.schadekaart import maak_schadekaart, maak_schade_polygon, bereken_schade_per_peilgebied_parallel, maak_schade_polygon_parallel
# import functions.wsa_tools as wsa #general tools used across all scripts
# from functions.select_polder_revision import select_polder_revision
# # from functions.API_start_calculation_gui_settings import * #ugly way of importing settings.
#Local imports
import sys
# sys.path.append(r'C:/Users/chris.kerklaan/Documents/Github/hhnk-threedi-tools')
sys.path.append('C:\\Users\wvangerwen\github\hhnk-threedi-tools')

from hhnk_threedi_tools import Folders
import hhnk_research_tools as hrt

import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline







In [2]:
#Purmerend
# --------------------------------------------------
polders_folder = "C:/Users/wvangerwen/Github/hhnk-threedi-tools/hhnk_threedi_tools/tests/data/multiple_polders"
polder = 'poldera' #Leeg laten voor handmatige invoer
# --------------------------------------------------
# BATCH_FOLDER_NAME = 'batch_Egmond_WO_juni21#4_sc2'


folder = Folders(os.path.join(polders_folder, polder))

import ipywidgets as widgets

def item_layout(width="95%", grid_area="", **kwargs):
    return widgets.Layout(
        width=width, grid_area=grid_area, **kwargs
    )  # override the default width of the button to 'auto' to let the button grow
    
output_folder_options = folder.threedi_results.batch.content
# output_folder_label = widgets.Label('Selecteer output folder:', layout=item_layout(grid_area='output_folder_label'))
output_folder_box = widgets.Select(
    options=output_folder_options,
    rows=3,
    disabled=False,
    layout=item_layout(grid_area="output_folder_box"),
)

print("Selecteer map met batch resultaten")
output_folder_box


# inladen mappenstructuur
# polder, revision, revision_nr, output_folder, folders_dict = select_polder_revision(polder, 'placeholder#01')
# batch_folder = os.path.join(folders_dict['05_extreme_data']['folder'], BATCH_FOLDER_NAME)
# batch_fd = create_batch_folders_dict(batch_folder)


# polder_shape = gpd.read_file(folders_dict['01_data']['polygon'])

# df = pd.DataFrame(batch_fd['01_downloads']['names'], columns=['dl_name'])
# for dl_name in batch_fd['01_downloads']['names']:
#     df.loc[df['dl_name']==dl_name, 'max_depth'] = batch_fd['01_downloads'][dl_name]['totaal']['max_depth']
#     df.loc[df['dl_name']==dl_name, 'max_depth_plas'] = batch_fd['01_downloads'][dl_name]['plas']['max_depth']
#     df.loc[df['dl_name']==dl_name, 'max_depth_overlast'] = batch_fd['01_downloads'][dl_name]['overlast']['max_depth']
#     df.loc[df['dl_name']==dl_name, 'total_damage'] = batch_fd['01_downloads'][dl_name]['totaal']['total_damage']
#     df.loc[df['dl_name']==dl_name, 'total_damage_plas'] = batch_fd['01_downloads'][dl_name]['plas']['total_damage']
#     df.loc[df['dl_name']==dl_name, 'total_damage_overlast'] = batch_fd['01_downloads'][dl_name]['overlast']['total_damage']
    
# if not os.path.exists(batch_fd['02_output_rasters']['folder']): os.mkdir(batch_fd['02_output_rasters']['folder'])

Selecteer map met batch resultaten


Select(layout=Layout(grid_area='output_folder_box', width='95%'), options=('bwn_test #5 (1) klimaatsommen',), …

In [ ]:
batch_fd = folder.threedi_results.batch[output_folder_box.value]

polder_shape = gpd.read_file(folder.source_data.polder_polygon.path)



In [ ]:
dir(hrt)

In [ ]:
neerslag_list, neerslag_nodata, neerslag_meta = hrt.load_gdal_raster(folders_dict['polder']['scripts']['neerslagsterkte'], bands=3)
fig, ax = plt.subplots(figsize=(15, 15))
ax.imshow(neerslag_list, extent=neerslag_meta['bounds'])
polder_shape.plot(ax=ax, color='red')